---
---

# Database Structure

In [ ]:
import mysql.connector

db_name = 'Structure'
user = 'root'
password = '12345'
host = 'localhost'
port = 3306

config = {
    'user': user,
    'password': password,
    'host': host,
    'port': port
}

try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"(CREATE DATABASE IF NOT EXISTS {db_name}) is ready.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    conn.close()

print(f"config = {config}")


---

- db.connect

In [ ]:
from peewee import *
db = MySQLDatabase(database=db_name,
                   user=user,
                   password=password,
                   host=host,
                   port=port)

db.connect()
print(f"Connected to the DB '{db.database}' successfully.")

---

- Model

In [ ]:
from peewee import Model

class BaseModel(Model):
    class Meta:
        database = db
        
print(f"{BaseModel} successfully.")

---

## **Category Table**: Contains a single column for `name`.

In [ ]:
from peewee import CharField

class Category(BaseModel):
    name = CharField(unique=True)
        
print(f"{Category} successfully.")

---

## **Client Table**: Contains columns for `first_name`, `last_name`, `phone`, `address`, and `email`.

In [ ]:
from peewee import CharField

# Assuming BaseModel and necessary imports are already defined
class Client(BaseModel):
    first_name = CharField()
    last_name = CharField()
    phone = CharField()
    address = CharField()
    email = CharField()
    
print(f"{Client} successfully.")

---

## - **Project Table**: Contains columns for 
- `name`,
- `duration` (in days),
- and foreign keys to `client` 
- and `category`.


In [ ]:
from peewee import CharField, IntegerField, ForeignKeyField

class Project(BaseModel):
    name = CharField()
    duration = IntegerField()
    client = ForeignKeyField(Client, backref='projects')
    category = ForeignKeyField(Category, backref='projects')

print(f"{Project} successfully.")

---

- create_tables

In [ ]:
db.connect
db.create_tables([Category, Client, Project])

table_names = db.get_tables()
print(f"create_tables{table_names} successfully.")

---

---

# **Data Insertion**:

## - Insert sample data into `client`, `category`, and `project` tables.

### Category

In [ ]:
from faker import Faker
fake = Faker()

category1 = Category.create(name=fake.name())


print(f"Insert {category1.name} successfully.")

### Client

In [ ]:
from faker import Faker
fake = Faker()
client1 = Client.create(
                first_name=fake.name(),
                last_name=fake.name(),
                phone=fake.phone_number(),
                address=fake.address(),
                email=fake.email()
                )
print(f"Insert {client1.first_name} {client1.last_name} {client1.phone} {client1.address} {client1.email} successfully.")

### Project

In [ ]:
from faker import Faker
fake = Faker()
import random
project1 = Project.create(
                  name=fake.company(),
                  duration=random.randint(1, 365),
                  client=client1,
                  category=category1
                  )
print(f"Insert {project1.name} {project1.duration} {project1.client} {project1.category}  successfully.")

---

# . **Dummy Data**:
   - Create dummy data (
      - 20 projects,
      - 5 categories,
      - 20 clients) 
      using external sources (possibly a website) and save them as JSON or CSV files.

## csv Category

### Dummy Data

In [ ]:
import csv
from faker import Faker

fake = Faker()

categories = [] 

def wcsv():
    for _ in range(2,7): 
        data = {
            "name": fake.name(),
        }
        categories.append(data) 
wcsv()
   
with open('categories.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["name"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in categories:
        writer.writerow(item)
    
print(f"{categories}Data has been written to data.csv")


### Insert

In [ ]:
try:
    with open('categories.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            try:
        
                Category.create(name=row['name'])
            except IntegrityError:
        
                print(f"Category '{row['name']}' already exists.")
    print(f"Total categories: {Category.select().count()}")
except Exception as e:
    print(f"An error occurred: {e}")

## csv Client

### Dummy Data

In [ ]:
import csv
from faker import Faker

fake = Faker()

clients = [] 

def wcsv():
    for _ in range(2,22): 
        data = {
            'first_name':fake.name(),
            'last_name':fake.name(),
            'phone':fake.phone_number(),
            'address':fake.address(),
            'email':fake.email()
        }
        clients.append(data) 
wcsv()
   
with open('clients.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['first_name','last_name','phone','address','email']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in clients:
        writer.writerow(item)
    
print(f"{clients}Data has been written to data.csv")



### Insert

In [ ]:
with open('clients.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:

        Client.create(
            first_name=row['first_name'],
            last_name=row['last_name'],
            phone=row['phone'],
            address=row['address'],
            email=row['email']
        )
print(f"{Client.select().count()}")

## csv Project

In [ ]:
import csv
from peewee import *
from faker import Faker
import random

fake = Faker()

projects = []

def wcsv():
    
        
    client_count = Client.select().count()
    category_count = Category.select().count()
    
    for _ in range(20):  

       
        client = Client.select().offset(random.randint(0, client_count - 1)).first()
        category = Category.select().offset(random.randint(0, category_count - 1)).first()

  
        data = {
            'name': fake.company(),  
            'duration': random.randint(1, 365),  
            'client_id': client.id,
            'category_id': category.id
        }
        projects.append(data)


wcsv()


with open('projects.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['name', 'duration', 'client_id', 'category_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in projects:
        writer.writerow(item)

print("Data has been written to projects.csv")


---

# . **Loading Data**:
   - Load the dummy data into the database tables.

- Insert

In [ ]:
with open('projects.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:

        Project.create(
            name=row['name'],
            duration=row['duration'],
            client=row['client_id'],
            category=row['category_id']
        )
print(f"{Project.select().count()}")

---

# answer this questions

In [ ]:
select_all=Project.select()
for project in projects:  
    print(project)

---

## - how many project for each client

In [ ]:
from peewee import  fn
query = (Project
         .select(Client.first_name, fn.COUNT(Project.id).alias('project_count'))
         .join(Client)
         .group_by(Client.first_name))

for result in query:
    print(f'Client Name: {result.client.first_name}, Project Count: {result.project_count}')

num_clients_with_projects = query.count()
print(f'Number of clients with projects: {num_clients_with_projects}')


---

## - how many project in each category

In [ ]:
from peewee import  fn
query = (Project
         .select(Category.name, fn.COUNT(Project.id).alias('project_count'))
         .join(Category)
         .group_by(Category.name))

for result in query:
    print(f'Category Name: {result.category.name}, Project Count: {result.project_count}')

num_category_with_projects = query.count()
print(f'Number of categorys with projects: {num_category_with_projects}')

---

## - how many project we have with duaration > 30 day

In [ ]:
duaration = (Project
            .select()
            .where(Project.duration > 30))

for x in duaration:
    print(x.name, x.duration)

---

## - what is the first project to finish

In [ ]:
first_finish = (Project
                .select()
                .order_by(Project.duration.asc())
                .first())
print(first_finish.name, first_finish.duration)

---

## - what is the last project to finish

In [ ]:
last_finish = (Project
               .select()
               .order_by(Project.duration.desc())
               .first())
print(last_finish.name, last_finish.duration)

---

 - connection closed

In [ ]:
if not db.is_closed():
    db.close()
    print(f"DB {db_name} connection closed.")
else:
    print("There are no open databases.")
    

---